<a href="https://colab.research.google.com/github/Sritam33/Deep_learning/blob/main/titanic_data_mlp_using_dropout_layer_%2Bearly_stoping_%2Blr_scheduler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )
device

device(type='cuda')

In [ ]:
training_data = pd.read_csv("/content/train (2).csv")
test_data = pd.read_csv("/content/test (2).csv")
print(training_data.shape)
print(test_data.shape)

(891, 12)
(418, 11)


In [ ]:
training_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
training_data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [ ]:
def preprocessing(data,is_train = True):
  # drop unnecessary features
  data.drop(columns= ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)
  data["Age"].fillna(data["Age"].mean(),inplace = True)
  data["Embarked"].fillna(data["Embarked"].mode()[0],inplace= True)
  data["Fare"].fillna(data["Fare"].mean(),inplace = True)

  # encode categorical feature
  label_encoder = LabelEncoder()
  data["Sex"] = label_encoder.fit_transform(data["Sex"])
  data["Embarked"] = label_encoder.fit_transform(data["Embarked"])

  # select features
  columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

  if is_train:
    X = data[columns].values
    y = data["Survived"].values
    return X,y
  else:
    X = data[columns].values
    return X


In [ ]:
X_train,y_train = preprocessing(training_data)
X_test = preprocessing(test_data,is_train=False)

X_train,X_valid,y_train,y_valid = train_test_split(X_train,y_train,test_size=0.2,random_state = 9)



In [ ]:
# convert data to pytorch tensors

X_train_tensor = torch.tensor(X_train,dtype = torch.float32)
y_train_tensor = torch.tensor(y_train,dtype = torch.long)
X_val_tensor = torch.tensor(X_valid,dtype = torch.float32)
y_val_tensor = torch.tensor(y_valid,dtype = torch.long)
X_test_tensor = torch.tensor(X_test,dtype = torch.float32)


In [ ]:
# create data loader
train_dataset = TensorDataset(X_train_tensor,y_train_tensor)
val_dataset = TensorDataset(X_val_tensor,y_val_tensor)

batch_size = 64

train_dataloader = DataLoader(train_dataset,batch_size = 64,shuffle=True)
val_dataloader = DataLoader(val_dataset,batch_size = 64)
test_dataloader = DataLoader(X_test_tensor,batch_size = 64)

In [ ]:
#Define the nueral network with dropout

class TitanicNN(nn.Module):
  def __init__(self,input_size):
    super(TitanicNN, self).__init__()
    self.layer1 = nn.Linear(input_size,64)
    self.relu = nn.ReLU()
    self.dropout = nn.Dropout(0.5)
    self.layer2 = nn.Linear(64,2)
    self.softmax = nn.Softmax(dim=1)

  def forward(self,x):
    x = self.layer1(x)
    x = self.relu(x)
    x = self.dropout(x)
    x = self.layer2(x)
    x = self.softmax(x)
    return x

In [ ]:
# model making
input_size = X_train.shape[1]
model = TitanicNN(input_size).to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.001)

In [ ]:
# training phase
# importing learning rate scheduler
import torch.optim.lr_scheduler as lr_scheduler
scheduler = lr_scheduler.StepLR(optimizer,step_size=20,gamma = 0.1)# creating step lr scheduler
# setting up early stopping parameters
patience = 10
best_loss = float("inf")
counter = 0
epoch = 100
val_losses = []
train_losses=[]
# model training starts
for epoch in range(epoch):
  model.train()
  train_loss = [] # Reset the train_loss list for each epoch
  for batch in train_dataloader:
    X,y = batch
    X,y = X.to(device), y.to(device)
    output = model(X)
    optimizer.zero_grad()
    loss = loss_fn(output,y)
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())

  model.eval()
  correct = 0
  total = 0
  val_losses = [] # Reset the val_losses list for each epoch
  with torch.no_grad():
    for batch in val_dataloader:
      X,y = batch
      X,y = X.to(device), y.to(device)
      output = model(X)
      loss  = loss_fn(output,y)
      val_losses.append(loss.item())

      _, predicted = torch.max(output.data,1)
      total = total + y.size(0)
      correct += (predicted ==y).sum().item()
    accuracy = correct * 100 / total # accuracy of validation in one epoch

  # step the learning rate scheduler()
  scheduler.step() # it will update the learning rate

  print("epoch: {}, train loss: {:.4f}, validation loss: {:.4f},validation accuracy:{:.3f}%".format
                 (epoch + 1, np.mean(train_loss), np.mean(val_losses), accuracy))

  if np.mean(val_losses) < best_loss :
    best_loss = np.mean(val_losses)
    counter = 0
  else:
    counter += 1

  if counter >= patience :
    print(f"early stopping after {epoch + 1} epochs")
    break




epoch: 1, train loss: 0.7133, validation loss: 0.6884,validation accuracy:60.335%
epoch: 2, train loss: 0.6398, validation loss: 0.6933,validation accuracy:60.894%
epoch: 3, train loss: 0.6452, validation loss: 0.6946,validation accuracy:60.335%
epoch: 4, train loss: 0.6382, validation loss: 0.6959,validation accuracy:60.335%
epoch: 5, train loss: 0.6241, validation loss: 0.6972,validation accuracy:60.335%
epoch: 6, train loss: 0.6278, validation loss: 0.6970,validation accuracy:60.894%
epoch: 7, train loss: 0.6431, validation loss: 0.6958,validation accuracy:60.335%
epoch: 8, train loss: 0.6216, validation loss: 0.6969,validation accuracy:60.894%
epoch: 9, train loss: 0.6359, validation loss: 0.6925,validation accuracy:60.894%
epoch: 10, train loss: 0.6297, validation loss: 0.6865,validation accuracy:61.453%
epoch: 11, train loss: 0.6304, validation loss: 0.6829,validation accuracy:62.570%
epoch: 12, train loss: 0.6468, validation loss: 0.6798,validation accuracy:62.570%
epoch: 13, tr

In [ ]:
# prediction for test data

model.eval()
prediction = []
with torch.no_grad():
  for batch in test_dataloader:
    X = batch
    X = X.to(device)
    output = model(X)
    _,predicted = torch.max(output.data,1)
    prediction.extend(predicted.cpu().numpy())
    total = total + X.size(0)

In [ ]:
len(prediction)

418

In [ ]:
total

597

In [ ]:
X_test.shape

(418, 7)

In [ ]:
df = pd.DataFrame(prediction)
df.value_counts()


0    306
1    112
dtype: int64